In [1]:
#!/usr/bin/env python
import sys
from pathlib import Path

from ase.io import read, write
from ase.spacegroup import get_spacegroup

def isequal_species(species1:list[str], species2:list[str]):
    """判断species1中全部元素是否都在species2中"""
    if len(species1) == len(species2):
        for spe in species1:
            if spe not in species2:
                return False
        else:
            return True

# species = sys.argv[1:]
species = ['Ce', 'Al']
# 获取当前目录下的所有文件
current_dir = Path('../test')

# 筛选出以 .vasp 结尾的文件名
vasp_files = [file for file  in current_dir.iterdir() if file.suffix == '.vasp']

i = 0
for vf in vasp_files:
    at = read(vf)
    if isequal_species(species1=species, species2=set(at.symbols)):
        i += 1
        newname = current_dir.joinpath(str(i)+'.'+f'{at.symbols}' + '.vasp')
        write(filename=newname, images=at, format='vasp')
    else:
        print(False)


## 写lammps文件结构

In [3]:
from ase.io import read, write
from ase.io.lammpsdata import write_lammps_data
from ase import Atoms
import numpy as np

def remove_hydrogen_and_convert(input_file, output_file, element_order):
    """
    读取 LAMMPS 轨迹文件，删除氢原子，按照指定顺序重新排列元素，写入 LAMMPS 格式文件。
    
    Args:
        input_file (str): 输入的 LAMMPS trajectory 文件名。
        output_file (str): 输出的 LAMMPS 格式文件名。
        element_order (list): 指定的元素排列顺序，例如 ['C', 'O', 'H', 'N']。
    """
    # 读取 LAMMPS 轨迹
    frames = read(input_file, format='lammps-dump-text', index=':')
    print(f"读取到 {len(frames)} 帧轨迹")

    # 处理每一帧，删除氢原子并按照指定顺序排列元素
    processed_frames = []
    for i, frame in enumerate(frames):
        symbols = frame.get_chemical_symbols()
        # 只保留非氢和非铍原子
        indices_to_keep = [i for i, symbol in enumerate(symbols) if symbol not in ['H', 'Be']]
        
        # 删除氢和铍原子
        new_frame = frame[indices_to_keep]
        
        # 按照指定的元素顺序排序
        element_to_index = {element: idx for idx, element in enumerate(element_order)}
        
        # 获取排序后的索引
        sorted_indices = sorted(range(len(new_frame)), key=lambda i: element_to_index[new_frame.get_chemical_symbols()[i]])
        
        # 根据指定顺序排列原子
        new_frame = new_frame[sorted_indices]

        processed_frames.append(new_frame)
        # print(f"帧 {i+1}: 保留 {len(new_frame)} 个原子")
    # print(len(processed_frames))
    # write(output_file, processed_frames, format='vasp-xdatcar')
    
    # 将结果写入 LAMMPS 格式文件
    # write(output_file, processed_frames, format='lammps-data')

    write(output_file, processed_frames, format='lammps-dump-text')
    # write_lammps_data(output_file, processed_frames[0], specorder=['La', 'Y', 'Ce', 'Th'])
    print(f"转换完成，输出文件为 {output_file}")

# 设置输入输出文件
input_file = "../test/A.lammpstrj"
output_file = "../test/new.lammpstrj"

# 定义元素排列顺序
element_order = ['La', 'Y', 'Ce', 'Th']  # 按照这个顺序排列元素

# 执行转换
remove_hydrogen_and_convert(input_file, output_file, element_order)


读取到 131 帧轨迹


ValueError: Can't write to lammps-dump-text-format